In [1]:
import pandas as pd
import numpy as np
import json

# 시각화 라이브러리
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import koreanize_matplotlib
import folium
from folium.plugins import MarkerCluster

In [2]:
# 그래프에 retina display 적용
%config InlineBackend.figure_format = 'retina'

In [3]:
file_path = './data/'

# 데이터 불러오기

In [4]:
df_voice_phising = pd.read_csv(f'{file_path}voice_phising.csv')
df_daejeon = pd.read_csv(f'{file_path}daejeon_population.csv')
df_sejong = pd.read_csv(f'{file_path}sejong_population.csv')
df_chungnam = pd.read_csv(f'{file_path}chungnam_population.csv')

In [5]:
df_voice_phising.head()

,접수부서,접수완료일시,경찰청구분,사건상태,사건종류,신고자 성별,발생지점,발생지 위도,발생지 경도,동일사건여부,연도,월,연월,요일,시간대,일자,발생시도,발생시군구
0,충남청,2020-01-02 09:33:49,충남청,종결요청,보이스피싱,불상,충청남도 천안시 서북구 차암동(부성2동) 524,127.104436,36.841583,NaN,2020.0,1.0,2020-01,목,9.0,2020-01-02,충청남도,천안시
1,충남청,2020-01-02 09:42:45,충남청,종결요청,보이스피싱,불상,충청남도 아산시 풍기동(행정:온양6동) 404-33,127.021875,36.771208,NaN,2020.0,1.0,2020-01,목,9.0,2020-01-02,충청남도,아산시
2,충남청,2020-01-02 10:23:29,충남청,종결요청,보이스피싱,불상,충청남도 서산시 대산읍 대산리(행정:대산읍) 688-1,126.426100,36.947617,NaN,2020.0,1.0,2020-01,목,10.0,2020-01-02,충청남도,서산시
3,대전청,2020-01-02 10:46:31,대전청,종결요청,보이스피싱,불상,대전광역시 유성구 대정동(행정:진잠동) 1-4,127.336373,36.317773,NaN,2020.0,1.0,2020-01,목,10.0,2020-01-02,대전광역시,유성구
4,충남청,2020-01-02 11:00:20,충남청,종결요청,보이스피싱,불상,충청남도 논산시 내동(행정:취암동) 1113,127.104068,36.182762,NaN,2020.0,1.0,2020-01,목,11.0,2020-01-02,충청남도,논산시


In [6]:

display(df_daejeon, df_chungnam, df_sejong)

,지역별,2020,2021
0,동구,232559,233721
1,대덕구,176393,175734
2,유성구,368895,367288
3,서구,478629,475671
4,중구,231959,227326


,지역별,2020,2021
0,계룡시,40854,41415
1,당진시,168955,169507
2,청양군,30103,30103
3,논산시,119707,117262
4,예산군,77838,76490
5,홍성군,102757,101618
6,보령시,99088,97390
7,태안군,60901,60285
8,부여군,64207,62415
9,천안시,682199,683888


,지역별,2020,2021
0,조치원읍,44462,44712
1,연기면,2835,2896
2,연동면,3625,3418
3,부강면,6479,6369
4,금남면,9202,9269
5,장군면,7229,7467
6,연서면,7693,7560
7,전의면,6078,5903
8,전동면,3731,3523
9,소정면,2634,2462


In [7]:
df_daejeon.columns = ['지역별', '2020 인구', '2021 인구']
df_chungnam.columns = ['지역별', '2020 인구', '2021 인구']
df_sejong.columns = ['지역별', '2020 인구', '2021 인구']

## 발생시도별 보이스피싱 데이터 분류

In [8]:
df_vp_daejeon_20 = df_voice_phising[(df_voice_phising['발생시도'] == '대전광역시') & (df_voice_phising['연도'] == 2020.0)]
df_vp_daejeon_21 = df_voice_phising[(df_voice_phising['발생시도'] == '대전광역시') & (df_voice_phising['연도'] == 2021.0)]
df_vp_chungnam_20 = df_voice_phising[(df_voice_phising['발생시도'] == '충청남도') & (df_voice_phising['연도'] == 2020.0)]
df_vp_chungnam_21 = df_voice_phising[(df_voice_phising['발생시도'] == '충청남도') & (df_voice_phising['연도'] == 2021.0)]
df_vp_sejong_20 = df_voice_phising[(df_voice_phising['발생시도'] == '세종시') & (df_voice_phising['연도'] == 2020.0)]
df_vp_sejong_21 = df_voice_phising[(df_voice_phising['발생시도'] == '세종시') & (df_voice_phising['연도'] == 2021.0)]

In [9]:
df_vp_sejong_20.loc[df_vp_sejong_20['발생시군구'] != '조치원읍', '발생시군구'] = df_vp_sejong_20.loc[df_vp_sejong_20['발생시군구'] != '조치원읍', '발생시군구'].apply(lambda x : x[:3])
df_vp_sejong_21.loc[df_vp_sejong_21['발생시군구'] != '조치원읍', '발생시군구'] = df_vp_sejong_21.loc[df_vp_sejong_21['발생시군구'] != '조치원읍', '발생시군구'].apply(lambda x : x[:3])

In [10]:
# 세종시 인구데이터가 있는 곳의 보이스피싱데이터만 가져오기
df_vp_sejong_20 = df_vp_sejong_20[df_vp_sejong_20['발생시군구'].isin(df_sejong['지역별'])]
df_vp_sejong_21 = df_vp_sejong_21[df_vp_sejong_21['발생시군구'].isin(df_sejong['지역별'])]

# 지역별 보이스피싱 데이터프레임 생성

## 대전광역시

In [11]:
df_vp_daejeon_20 = pd.DataFrame(df_vp_daejeon_20['발생시군구'].value_counts())
df_vp_daejeon_20 = df_vp_daejeon_20.reset_index()
df_vp_daejeon_20.columns = ['지역별', '2020']

df_vp_daejeon_21 = pd.DataFrame(df_vp_daejeon_21['발생시군구'].value_counts())
df_vp_daejeon_21 = df_vp_daejeon_21.reset_index()
df_vp_daejeon_21.columns = ['지역별', '2021']

In [12]:
display(df_vp_daejeon_20.head(), df_vp_daejeon_21.head())

,지역별,2020
0,서구,1009
1,유성구,837
2,동구,511
3,중구,509
4,대덕구,374


,지역별,2021
0,서구,899
1,유성구,765
2,중구,415
3,동구,385
4,대덕구,299


In [13]:
df_vp_daejeon = pd.merge(df_vp_daejeon_20, df_vp_daejeon_21, on='지역별', how='outer')
df_vp_daejeon

,지역별,2020,2021
0,서구,1009,899
1,유성구,837,765
2,동구,511,385
3,중구,509,415
4,대덕구,374,299


In [14]:
df_vp_daejeon.columns = ['지역별', '2020 보이스피싱', '2021 보이스피싱']

## 충청남도

In [15]:
df_vp_chungnam_20 = pd.DataFrame(df_vp_chungnam_20['발생시군구'].value_counts())
df_vp_chungnam_20 = df_vp_chungnam_20.reset_index()
df_vp_chungnam_20.columns = ['지역별', '2020']

df_vp_chungnam_21 = pd.DataFrame(df_vp_chungnam_21['발생시군구'].value_counts())
df_vp_chungnam_21 = df_vp_chungnam_21.reset_index()
df_vp_chungnam_21.columns = ['지역별', '2021']

In [16]:
display(df_vp_chungnam_20.head(), df_vp_chungnam_21.head())

,지역별,2020
0,천안시,2263
1,아산시,1063
2,당진시,586
3,서산시,571
4,공주시,371


,지역별,2021
0,천안시,3525
1,아산시,1700
2,당진시,931
3,서산시,874
4,논산시,518


In [17]:
df_vp_chungnam = pd.merge(df_vp_chungnam_20, df_vp_chungnam_21, on='지역별', how='outer')
df_vp_chungnam

,지역별,2020,2021
0,천안시,2263,3525
1,아산시,1063,1700
2,당진시,586,931
3,서산시,571,874
4,공주시,371,468
5,논산시,357,518
6,보령시,260,386
7,홍성군,253,344
8,예산군,239,325
9,태안군,188,235


In [18]:
df_vp_chungnam.columns = ['지역별', '2020 보이스피싱', '2021 보이스피싱']

## 세종특별자치시

In [19]:
df_vp_sejong_20 = pd.DataFrame(df_vp_sejong_20['발생시군구'].value_counts())
df_vp_sejong_20 = df_vp_sejong_20.reset_index()
df_vp_sejong_20.columns = ['지역별', '2020']

df_vp_sejong_21 = pd.DataFrame(df_vp_sejong_21['발생시군구'].value_counts())
df_vp_sejong_21 = df_vp_sejong_21.reset_index()
df_vp_sejong_21.columns = ['지역별', '2021']

In [20]:
display(df_vp_sejong_20.head(), df_vp_sejong_21.head())

,지역별,2020
0,조치원읍,288
1,아름동,85
2,보람동,63
3,한솔동,56
4,고운동,55


,지역별,2021
0,조치원읍,275
1,아름동,116
2,도담동,99
3,고운동,97
4,한솔동,91


In [21]:
df_vp_sejong = pd.merge(df_vp_sejong_20, df_vp_sejong_21, on='지역별', how='outer')
df_vp_sejong

,지역별,2020,2021
0,조치원읍,288,275
1,아름동,85,116
2,보람동,63,87
3,한솔동,56,91
4,고운동,55,97
5,도담동,51,99
6,다정동,50,78
7,종촌동,50,78
8,새롬동,50,73
9,소담동,38,63


In [22]:
df_vp_sejong.columns = ['지역별', '2020 보이스피싱', '2021 보이스피싱']

# 2020, 2021 인구수 대비 보이스피싱 피해 비율

In [23]:
# 인구수 데이터와 보이스피싱 신고 데이터 합치기
df_dj = pd.merge(df_daejeon, df_vp_daejeon, on='지역별', how='outer')
df_cn = pd.merge(df_chungnam, df_vp_chungnam, on='지역별', how='outer')
df_sj = pd.merge(df_sejong, df_vp_sejong, on='지역별', how='outer')

In [24]:
display(df_dj.head(), df_cn.head(), df_sj.head())

,지역별,2020 인구,2021 인구,2020 보이스피싱,2021 보이스피싱
0,동구,232559,233721,511,385
1,대덕구,176393,175734,374,299
2,유성구,368895,367288,837,765
3,서구,478629,475671,1009,899
4,중구,231959,227326,509,415


,지역별,2020 인구,2021 인구,2020 보이스피싱,2021 보이스피싱
0,계룡시,40854,41415,105,228
1,당진시,168955,169507,586,931
2,청양군,30103,30103,65,55
3,논산시,119707,117262,357,518
4,예산군,77838,76490,239,325


,지역별,2020 인구,2021 인구,2020 보이스피싱,2021 보이스피싱
0,조치원읍,44462,44712,288,275
1,연기면,2835,2896,14,23
2,연동면,3625,3418,14,23
3,부강면,6479,6369,32,36
4,금남면,9202,9269,32,60


In [25]:
df_dj['2020 보이스피싱 피해 비율'] = df_dj['2020 보이스피싱']/df_dj['2020 인구']*100
df_dj['2021 보이스피싱 피해 비율'] = df_dj['2021 보이스피싱']/df_dj['2021 인구']*100

df_dj

,지역별,2020 인구,2021 인구,2020 보이스피싱,2021 보이스피싱,2020 보이스피싱 피해 비율,2021 보이스피싱 피해 비율
0,동구,232559,233721,511,385,0.219729,0.164726
1,대덕구,176393,175734,374,299,0.212027,0.170144
2,유성구,368895,367288,837,765,0.226894,0.208283
3,서구,478629,475671,1009,899,0.210810,0.188996
4,중구,231959,227326,509,415,0.219435,0.182557


In [26]:
df_dj.to_csv(f'{file_path}daejeon_vp_percent.csv', index=False)

In [27]:
df_cn['2020 보이스피싱 피해 비율'] = df_cn['2020 보이스피싱']/df_cn['2020 인구']*100
df_cn['2021 보이스피싱 피해 비율'] = df_cn['2021 보이스피싱']/df_cn['2021 인구']*100

df_cn

,지역별,2020 인구,2021 인구,2020 보이스피싱,2021 보이스피싱,2020 보이스피싱 피해 비율,2021 보이스피싱 피해 비율
0,계룡시,40854,41415,105,228,0.257013,0.550525
1,당진시,168955,169507,586,931,0.346838,0.549240
2,청양군,30103,30103,65,55,0.215925,0.182706
3,논산시,119707,117262,357,518,0.298228,0.441746
4,예산군,77838,76490,239,325,0.307048,0.424892
5,홍성군,102757,101618,253,344,0.246212,0.338523
6,보령시,99088,97390,260,386,0.262393,0.396345
7,태안군,60901,60285,188,235,0.308698,0.389815
8,부여군,64207,62415,145,203,0.225832,0.325242
9,천안시,682199,683888,2263,3525,0.331721,0.515435


In [28]:
df_cn.to_csv(f'{file_path}chungcheong_vp_percent.csv', index=False)

In [29]:
df_sj['2020 보이스피싱']

0     288
1      14
2      14
3      32
4      32
5      14
6      18
7      29
8       7
9       5
10     56
11     50
12     51
13      1
14     85
15     50
16     55
17     38
18     32
19     63
20     34
21     50
Name: 2020 보이스피싱, dtype: int64

In [30]:
df_sj = df_sj.replace({'-' : np.nan})
df_sj = df_sj.dropna()

In [31]:
df_sj['2020 인구'] = df_sj['2020 인구'].astype('int')

In [32]:
df_sj['2020 보이스피싱 피해 비율'] = df_sj['2020 보이스피싱']/df_sj['2020 인구']*100
df_sj['2021 보이스피싱 피해 비율'] = df_sj['2021 보이스피싱']/df_sj['2021 인구']*100

df_sj

,지역별,2020 인구,2021 인구,2020 보이스피싱,2021 보이스피싱,2020 보이스피싱 피해 비율,2021 보이스피싱 피해 비율
0,조치원읍,44462,44712,288,275,0.647744,0.615047
1,연기면,2835,2896,14,23,0.493827,0.794199
2,연동면,3625,3418,14,23,0.386207,0.672908
3,부강면,6479,6369,32,36,0.493903,0.565238
4,금남면,9202,9269,32,60,0.347750,0.647319
5,장군면,7229,7467,14,37,0.193664,0.495514
6,연서면,7693,7560,18,26,0.233979,0.343915
7,전의면,6078,5903,29,27,0.477131,0.457395
8,전동면,3731,3523,7,23,0.187617,0.652853
9,소정면,2634,2462,5,14,0.189825,0.568643


In [33]:
for idx in list(df_sj.index):
    a = df_sj.loc[idx, '지역별']
    df_sj.loc[idx, '지역별'] = f'세종특별자치시 세종시 {a}'

In [34]:
df_sj

,지역별,2020 인구,2021 인구,2020 보이스피싱,2021 보이스피싱,2020 보이스피싱 피해 비율,2021 보이스피싱 피해 비율
0,세종특별자치시 세종시 조치원읍,44462,44712,288,275,0.647744,0.615047
1,세종특별자치시 세종시 연기면,2835,2896,14,23,0.493827,0.794199
2,세종특별자치시 세종시 연동면,3625,3418,14,23,0.386207,0.672908
3,세종특별자치시 세종시 부강면,6479,6369,32,36,0.493903,0.565238
4,세종특별자치시 세종시 금남면,9202,9269,32,60,0.347750,0.647319
5,세종특별자치시 세종시 장군면,7229,7467,14,37,0.193664,0.495514
6,세종특별자치시 세종시 연서면,7693,7560,18,26,0.233979,0.343915
7,세종특별자치시 세종시 전의면,6078,5903,29,27,0.477131,0.457395
8,세종특별자치시 세종시 전동면,3731,3523,7,23,0.187617,0.652853
9,세종특별자치시 세종시 소정면,2634,2462,5,14,0.189825,0.568643


In [35]:
df_sj.to_csv(f'{file_path}sejong_vp_percent.csv', index=False)

# choropleth 시각화

In [36]:
dj_geo = json.load(open(f'{file_path}geojson/TL_SCCO_SIG.json', encoding='utf-8'))
sj_geo = json.load(open(f'{file_path}geojson/hangjeongdong_세종특별자치시.geojson', encoding='utf-8'))

## 대전광역시

In [ ]:
dj_scale = (df_zip['cnt'].quantile((0, 0.2, 0.4, 0.6, 0.8, 1))).tolist()

### 2020

In [37]:
dj_20_m = folium.Map(
          location=[36.3504, 127.3845],
          zoom_start=11
)

folium.GeoJson(
    dj_geo,
    name='SIG_KOR_NM'
).add_to(dj_20_m)

dj_20_m.choropleth(geo_data=dj_geo,
                   data=df_dj, 
                   columns = ['지역별', '2020 보이스피싱 피해 비율'],
                   fill_color='YlOrRd', # 색상 변경도 가능하다
                   fill_opacity=0.5,
                   line_opacity=0.2,
                   key_on='properties.SIG_KOR_NM',
                   legend_name="대전광역시 2020 보이스피싱 비율"
                   )

dj_20_m.save('dj_20.html')

/Users/seunghoolee/miniconda/lib/python3.9/site-packages/folium/folium.py:465: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  warnings.warn(


### 2021

In [38]:
dj_21_m = folium.Map(
          location=[36.3504, 127.3845],
          zoom_start=11
)

folium.GeoJson(
    dj_geo,
    name='SIG_KOR_NM'
).add_to(dj_21_m)

dj_21_m.choropleth(geo_data=dj_geo,
                   data=df_dj, 
                   columns = ['지역별', '2021 보이스피싱 피해 비율'],
                   fill_color='YlOrRd', # 색상 변경도 가능하다
                   fill_opacity=0.5,
                   line_opacity=0.2,
                   key_on='properties.SIG_KOR_NM',
                   legend_name="대전광역시 2021 보이스피싱 비율"
                   )

dj_21_m.save('dj_21.html')

/Users/seunghoolee/miniconda/lib/python3.9/site-packages/folium/folium.py:465: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  warnings.warn(


## 충청남도

### 2020

In [39]:
cn_20_m = folium.Map(
          location=[36.5184, 126.8000],
          zoom_start=9
)

folium.GeoJson(
    dj_geo,
    name='SIG_KOR_NM'
).add_to(cn_20_m)

cn_20_m.choropleth(geo_data=dj_geo,
                   data=df_cn, 
                   columns = ['지역별', '2020 보이스피싱 피해 비율'],
                   fill_color='YlOrRd', # 색상 변경도 가능하다
                   fill_opacity=0.5,
                   line_opacity=0.2,
                   key_on='properties.SIG_KOR_NM',
                   legend_name="충청남도 2020 보이스피싱 비율"
                   )

cn_20_m.save('cn_20.html')

/Users/seunghoolee/miniconda/lib/python3.9/site-packages/folium/folium.py:465: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  warnings.warn(


### 2021

In [40]:
cn_21_m = folium.Map(
          location=[36.5184, 126.8000],
          zoom_start=9
)

folium.GeoJson(
    dj_geo,
    name='SIG_KOR_NM'
).add_to(cn_21_m)

cn_21_m.choropleth(geo_data=dj_geo,
                   data=df_cn, 
                   columns = ['지역별', '2021 보이스피싱 피해 비율'],
                   fill_color='YlOrRd', # 색상 변경도 가능하다
                   fill_opacity=0.5,
                   line_opacity=0.2,
                   key_on='properties.SIG_KOR_NM',
                   legend_name="충청남도 2021 보이스피싱 비율"
                   )

cn_21_m.save('cn_21.html')

/Users/seunghoolee/miniconda/lib/python3.9/site-packages/folium/folium.py:465: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  warnings.warn(


## 세종특별자치시

### 2020

In [41]:
sj_20_m = folium.Map(
          location=[36.5040736, 127.2494855],
          zoom_start=9
)

folium.GeoJson(
    sj_geo,
    name='adm_nm'
).add_to(sj_20_m)

sj_20_m.choropleth(geo_data=sj_geo,
                   data=df_sj, 
                   columns = ['지역별', '2020 보이스피싱 피해 비율'],
                   fill_color='YlOrRd', # 색상 변경도 가능하다
                   fill_opacity=0.5,
                   line_opacity=0.2,
                   key_on='properties.adm_nm',
                   legend_name="세종특별자치시 2020 보이스피싱 비율"
                   )

sj_20_m.save('sj_20.html')

/Users/seunghoolee/miniconda/lib/python3.9/site-packages/folium/folium.py:465: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  warnings.warn(


### 2021

In [42]:
sj_21_m = folium.Map(
          location=[36.5040736, 127.2494855],
          zoom_start=9
)

folium.GeoJson(
    sj_geo,
    name='adm_nm'
).add_to(sj_21_m)

sj_21_m.choropleth(geo_data=sj_geo,
                   data=df_sj, 
                   columns = ['지역별', '2021 보이스피싱 피해 비율'],
                   fill_color='YlOrRd', # 색상 변경도 가능하다
                   fill_opacity=0.5,
                   line_opacity=0.2,
                   key_on='properties.adm_nm',
                   legend_name="세종특별자치시 2021 보이스피싱 비율"
                   )

sj_21_m.save('sj_21.html')

/Users/seunghoolee/miniconda/lib/python3.9/site-packages/folium/folium.py:465: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  warnings.warn(
